In [8]:
ls ../experiments/emotions/data/

goemotions_1.csv  goemotions_2.csv  goemotions_3.csv


In [1]:
import pandas as pd
import glob
import os

full_df = []
for file in glob.glob('../experiments/emotions/data/*.csv'):
    df = pd.read_csv(file)
    full_df.append(df)
full_df = pd.concat(full_df)

In [29]:
file

'../experiments/emotions/data/goemotions_2.csv'

In [2]:
full_df.shape 

(211225, 37)

In [5]:
emotion_columns = ['admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']

full_df[emotion_columns].sum().sort_values(ascending=False)

neutral           55298
approval          17620
admiration        17131
annoyance         13618
gratitude         11625
disapproval       11424
curiosity          9692
amusement          9245
realization        8785
optimism           8715
disappointment     8469
love               8191
anger              8084
joy                7983
confusion          7359
sadness            6758
caring             5999
excitement         5629
surprise           5514
disgust            5301
desire             3817
fear               3197
remorse            2525
embarrassment      2476
nervousness        1810
pride              1302
relief             1289
grief               673
dtype: int64

In [10]:
full_df['text'].head().tolist()

['That game hurt.',
 ' >sexuality shouldn’t be a grouping category It makes you different from othet ppl so imo it fits the definition of "grouping" ',
 "You do right, if you don't care then fuck 'em!",
 'Man I love reddit.',
 '[NAME] was nowhere near them, he was by the Falcon. ']

In [17]:
from tqdm.auto import tqdm
tqdm.pandas()

In [19]:
emotions_condensed = full_df.progress_apply(lambda x: x[emotion_columns].loc[lambda s: s == 1].index.tolist(), axis=1)

  0%|          | 0/211225 [00:00<?, ?it/s]

In [20]:
emotions_condensed

0                      [sadness]
1                             []
2                      [neutral]
3                         [love]
4                      [neutral]
                  ...           
69995    [excitement, joy, love]
69996                  [neutral]
69997                  [disgust]
69998         [anger, annoyance]
69999               [admiration]
Length: 211225, dtype: object

In [21]:
data_df = full_df[['text', 'id'] + emotion_columns]

In [31]:
data_df.head(2)

,text,id,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
data_df.loc[lambda df: df[emotion_columns[0]] == 1][['text', emotion_columns[0]]]

,text,admiration
15,"I appreciate it, that's good to know. I hope I...",1
18,Pretty much every Punjabi dude I've met.,1
28,"Lots, play store or apple store vpn. Nord is good",1
35,nice!! I'll try this one,1
50,She’s like a kewpie doll with them. Precious.,1
...,...,...
69939,You better be!,1
69948,damn thats good,1
69957,Watch it!! It's amazing,1
69991,I like this response. I can’t give you gold bu...,1


In [27]:
data_df.to_csv('../experiments/emotions/data/combined_emotions.csv', index=False)

In [35]:
mv ../experiments/hate-speech/finegrained_reply.csv ../experiments/hate-speech/data/

In [39]:
hate_speech_df = pd.read_csv('../experiments/hate-speech/data/finegrained_reply.csv')

In [67]:
from unidecode import unidecode
# how to regex unescape 
import re 
def unescape(text):
    return re.sub(r'\\(.)', r'\1', text)

import html
import re

def clean_text(text):
    # Step 1: Unescape HTML entities
    text = html.unescape(text)
    
    # Step 2: Remove zero-width characters (like U+200B)
    text = re.sub(r'[\u200B\u200C\u200D\uFEFF]', '', text)

    # Step 3: Remove backslash-escaped characters
    text = re.sub(r'\\+([^\\])', r'\1', text)
    
    # Step 4: Clean up any remaining HTML entities
    text = re.sub(r'&[a-zA-Z]+;', '', text)
    text = re.sub(r'&[#a-zA-Z0-9]+;', '', text)
    
    # Step 5: Normalize whitespace (preserve newlines but clean up other whitespace)
    text = re.sub(r'[ \t]+', ' ', text)
    
    return text.strip()

In [68]:
hate_speech_df['Hate_body'].iloc[0]

'\\&gt;actually beliving the jewish iq meme\n\nyeah man ashkenazis are pretty smart. largely due to the fact that they posses high ammounts of european dna that they preserved through millenia of inbreeding.\n\n&amp;#x200B;\n\nkhazarians and the other nosed folk are borderline retarded'

In [76]:
hate_speech_df['Hate_body'] = hate_speech_df['Hate_body'].str.replace('\n', ' ').apply(clean_text)

In [77]:
hate_speech_df['Reply_body'] = hate_speech_df['Reply_body'].str.replace('\n', ' ').apply(clean_text)

In [79]:
hate_speech_df.to_csv('../experiments/hate-speech/data/finegrained_reply_cleaned.csv', index=False)

In [82]:
hate_speech_df['Hate_ID'].value_counts()

Hate_ID
hgjbnp0    41
gyek6xd    36
gnvk4mo    31
gw24xuz    24
h1546zi    21
           ..
hcxy9n2     1
gmwclya     1
he3hi1q     1
go3hv3v     1
h5dvew2     1
Name: count, Length: 1065, dtype: int64

In [84]:
hate_speech_df.head(2)

,Hate_ID,Reply_ID,Hate_body,Reply_body,Gold_label
0,ehjlqzo,ehk3o7u,>actually beliving the jewish iq meme yeah man...,"they fucking are, at least i think they are. i...",agree
1,ehjlqzo,ehkpe3e,>actually beliving the jewish iq meme yeah man...,So if inbreeding preserves that southern redne...,justification


In [49]:
print(hate_speech_df['Hate_body'].iloc[0])

\&gt;actually beliving the jewish iq meme

yeah man ashkenazis are pretty smart. largely due to the fact that they posses high ammounts of european dna that they preserved through millenia of inbreeding.

&amp;#x200B;

khazarians and the other nosed folk are borderline retarded


# Discourse

In [133]:
df = pd.read_json('../experiments/news-discourse/data/reparsed-newsworthiness-df.jsonl', lines=True)

In [134]:
df

,sentence,label,split
0,[FRANKFURT (Reuters) - German conglomerate Thy...,"[Main, Main, Main, Main, Main, Main_Consequenc...",train
1,[(Updates with valuation from source in paragr...,"[NA, Distant_Expectations_Consequences, Distan...",train
2,[NEW YORK (Reuters) - Global equity markets fe...,"[Main, Cause_General, NA, Distant_Evaluation, ...",train
3,[(Reuters) - Activist investor Daniel Loeb’s h...,"[Main, Cause_General, Main_Consequence, Cause_...",train
4,[(Reuters) - Health insurer Aetna Inc AET.N sa...,"[Main, Distant_Expectations_Consequences, Caus...",train
...,...,...,...
797,[MUTUAL fund managers are unfairly blamed for ...,"[Main, Distant_Evaluation, Distant_Evaluation,...",test
798,"[IF 2006 proved anything, it's that aging bull...","[Distant_Evaluation, Cause_General, Cause_Gene...",test
799,"[MERGERS and acquisitions have been booming, a...","[Main, Distant_Evaluation, Distant_Evaluation,...",test
800,"[IN the last three months, as Americans debate...","[Distant_Evaluation, Distant_Evaluation, Dista...",test


In [93]:
import glob
all_files = glob.glob('../experiments/news-discourse/data/*/**', recursive=True)


In [ ]:
txt_files = list(filter(lambda x: x.endswith('.txt'), all_files))
ann_files = list(filter(lambda x: x.endswith('.ann'), all_files))

In [111]:
text_file_df = (pd.Series(txt_files)
 .to_frame('text_file')
 .assign(split=lambda df: df['text_file'].apply(lambda x: 'train' if '/train/' in x else 'dev' if '/validation/' in x else 'test' if '/test/' in x else 'unknown'))
 .assign(file_name=lambda df: df['text_file'].apply(lambda x: x.split('/')[-1].replace('.txt', '')))
)




In [112]:
ann_file_df = (pd.Series(ann_files)
 .to_frame('ann_file')
 .assign(split=lambda df: df['ann_file'].apply(lambda x: 'train' if '/train/' in x else 'dev' if '/validation/' in x else 'test' if '/test/' in x else 'unknown'))
 .assign(file_name=lambda df: df['ann_file'].apply(lambda x: x.split('/')[-1].replace('.ann', '')))
)

In [118]:
full_file_df = ann_file_df[['ann_file', 'file_name']].merge(text_file_df, on='file_name', how='left')

In [ ]:
ann_file, _, text_file, _ = full_file_df.iloc[0]

In [126]:
ann_split = list(map(lambda x: x.split('\t'), open(ann_file).read().strip().split('\n')))

In [130]:
ann_split = list(filter(lambda x: x[0][0] =='T', ann_split))

In [132]:
open(text_file).read().strip().split('\n')

['H  GYULA HORN, WHO HELPED PART IRON CURTAIN, DIES AT 80 ',
 '',
 'S1  Gyula Horn, a former leader of Hungary who in 1989 ripped a hole in the Iron Curtain, helping to set off months of tumultuous change in which communist governments in Eastern Europe fell one after the other, died Wednesday in Budapest.',
 'S2 He was 80.',
 '',
 'S3  The Hungarian government announced the death.',
 'S4 He had been hospitalized since 2007 with what was reported to be a brain malfunction.',
 '',
 'S5  Horn’s life encompassed much of the history of 20th-century Hungary.',
 'S6 His father, a communist, was executed by the Nazis occupying Hungary in 1944.',
 'S7 Gyula (pronounced JOO-la) also became a hard-line communist, serving in militia units that hunted down government opponents during their revolt in 1956.',
 'S8 The rebels lynched his brother, also a communist.',
 '',
 'S9  As foreign minister, as Moscow’s grip on Eastern Europe slipped, Horn proved nimble as a newly minted, nonideological, pragma